# Generating Text using Prompt engineering
This notebook showcases text generation using various prompt engineering techniques

In [ ]:
#Ensure you have the latest version of boto3 & botocore packages 
%pip install boto3 botocore langchain-community

In [ ]:
#import the main packages and libraries
import os
import boto3
import json
import botocore
from langchain_community.llms import Bedrock

In [ ]:
bedrock_client = boto3.client('bedrock-runtime') # Provide the desired region

## Zero-shot Contextual prompting

In [ ]:
prompt = """
Human: Write a descriptive and engaging travel guide section about a lesser-known but beautiful destination, capturing the local culture, cuisine, and must-see attractions in a way that inspires wanderlust.
Assistant:"""

In [ ]:
body = json.dumps({"prompt": prompt,"max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"  # change this to use a different version from the model provider
accept = "application/json"
contentType = "application/json"

In [ ]:
# Invoking the model
response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get("body").read())

In [ ]:
print(response_body.get("completion"))

## Few-shot contextual prompting

In [ ]:
prompt = """
Human: Here are some examples of product descriptions:

Example 1:
Product: Apple iPhone 13 Pro
Description: The iPhone 13 Pro is a smartphone designed and manufactured by Apple Inc. It features a 6.1-inch Super Retina XDR display, a powerful A15 Bionic chip, and an advanced triple-camera system with improved low-light performance and 3x optical zoom. The phone also boasts 5G connectivity, longer battery life, and a durable Ceramic Shield front cover.

Example 2:
Product: Sony WH-1000XM4 Noise Cancelling Headphones
Description: Experience exceptional audio quality with the Sony WH-1000XM4 Noise Cancelling Headphones. These over-ear headphones feature industry-leading noise cancellation technology, allowing you to immerse yourself in your music without distractions. The responsive touch controls and long-lasting battery life make them ideal for everyday use, while the comfortable design ensures hours of listening pleasure.

Example 3:
Product: Instant Pot Duo Crisp + Air Fryer
Description: The Instant Pot Duo Crisp + Air Fryer is a versatile kitchen appliance that combines the functions of an electric pressure cooker, air fryer, and more. With its EvenCrisp technology, you can achieve crispy, golden results using little to no oil. The easy-to-use control panel and 11 built-in smart programs allow you to cook a wide variety of dishes with ease, making it a must-have for any modern kitchen.

Your task: Generate a product description for the following product:
Product: Sony A7 III Mirrorless Camera
Assistant:"""

In [ ]:
body = json.dumps({"prompt": prompt, "max_tokens_to_sample": 500})
modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

In [ ]:
response = bedrock_client.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get("body").read())

In [ ]:
print(response_body.get("completion"))

## Zero-shot prompting with LangChain

In [ ]:
inference_modifier = {"max_tokens_to_sample": 4096, "temperature": 0.5, "top_k": 250, "top_p": 1, "stop_sequences": ["\n\nHuman"],}

In [ ]:
llm = Bedrock(model_id="anthropic.claude-v2",  client=bedrock_client, model_kwargs=inference_modifier,)

In [ ]:
response = llm("""
Human: Write a descriptive and engaging travel guide section about a lesser-known but 
beautiful destination, capturing the local culture, cuisine, and must-see attractions 
in a way that inspires wanderlust.
Assistant:""")

In [ ]:
print(response)

## Contextual Generation with LangChain

In [ ]:
inference_modifier = {'max_tokens_to_sample':4096, "temperature":0.5, "top_k":250, "top_p":1, "stop_sequences": ["\n\nHuman"]}

In [ ]:
llm = Bedrock(model_id = "anthropic.claude-v2", client = bedrock_client, model_kwargs = inference_modifier)

In [ ]:
from langchain.prompts import PromptTemplate
product_description_prompt = PromptTemplate(   input_variables=["product_name", "product_category", "key_features"],
    template="""
You are a professional copywriter tasked with creating an engaging and informative product description for a new Amazon product.

Product Name: {product_name}
Product Category: {product_category}
Key Features: {key_features}

Write a compelling product description that highlights the key features and benefits of the product, while keeping the tone engaging and persuasive for potential customers.

Product Description:
"""
)

In [ ]:
prompt = product_description_prompt.format(
    product_name="Smart Home Security Camera",
    product_category="Home Security",
    key_features="- 1080p HD video recording\n- Motion detection alerts\n- Two-way audio communication\n- Night vision capabilities\n- Cloud storage for recorded footage")

In [ ]:
response = llm(prompt)
product = response[response.index('\n')+1:]

In [ ]:
print(product)